# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación

## Minería de Datos

### Curso académico 2021-2022

#### Profesorado:

* Juan Carlos Alfaro Jiménez
* José Antonio Gámez Martín

---

**Notas**:

* Adaptado de las prácticas de Jacinto Arias Martínez y Enrique González Rodrigo.

---

En esta práctica vamos a trabajar algunos de los aspectos más importantes del proceso *KDD* (*Knowledge Discovery from Data*):

* Almacenamiento y carga de datos
* Análisis exploratorio de datos
* Preprocesamiento de datos
* Validación de modelos de clasificación

Para ello, aprenderemos a manipular y visualizar los datos mediante distintas funciones de la librería `plotly`. Además, aprenderemos a utilizar algoritmos de clasificación como *Zero-R* y árboles de decisión usando la librería `scikit-learn`.

El objetivo de la práctica será aprender a cargar, explorar y preparar nuestros datos, aprender y validar distintos modelos de clasificación y ser capaces de interpretar los resultados obtenidos. Para lograrlo, utilizaremos dos conjuntos de datos sintéticos:

- `iris`: https://www.kaggle.com/uciml/iris
- `titanic`: https://www.kaggle.com/c/titanic

La descripción de cada uno de estos conjuntos de datos se encuentra en el enlace correspondiente.

# 1. Preliminares

Antes de comenzar, vamos a fijar una semilla para que los experimentos sean reproducibles:

In [1]:
random_state = 27912

Y definir una serie de funciones útiles que usaremos posteriormente:

In [2]:
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
from sklearn.utils.multiclass import unique_labels

In [4]:
import plotly.figure_factory as ff

In [5]:
def fit_evaluate(estimator, X_train, X_test, y_train, y_test):
    """Fit the estimator to the training dataset and evaluate in the testing dataset."""
    # Build the estimators from the training dataset
    estimator = estimator.fit(X_train, y_train)

    # Predict the classes for the testing dataset
    predictions = estimator.predict(X_test)

    # Build a text report (in a dictionary) showing the main classification metrics
    report = classification_report(y_test, predictions, output_dict=True)

    # Convert the report to a tabular representation
    report = pd.DataFrame(report).T

    # Compute the confusion matrix to evaluate the accuracy of a classification
    matrix = confusion_matrix(y_test, predictions)

    # Extract an ordered array of unique labels
    out = unique_labels(y_test, predictions)

    # Convert the unique labels to a list 
    x = y = list(out)

    # Create a figure with annotations to each cell of the heatmap
    matrix = ff.create_annotated_heatmap(matrix, x=x, y=y)

    # Define the label for the columns (predictions)
    xaxis = {"title": "Predicted label"}

    # Define the label for the rows (true classes) 
    yaxis = {"title": "True label"}

    # Update the confusion matrix to show the labels
    matrix.update_layout(xaxis=xaxis, yaxis=yaxis)

    return estimator, predictions, report, matrix

# 2. Acceso y almacenamiento de datos

Se trata de uno de los problemas críticos en cualquier aplicación real, ya que tendremos que tener en cuenta numerosas preguntas:

* ¿Dónde se generan los datos?
* ¿Con qué frecuencia?
* ¿Cuánto cuesta transmitirlos?
* ¿Dónde se van a almacenar?
* ¿Cuánto ocupan?

Además de añadir un alto grado de dificultad al proceso, las soluciones que aportemos suelen ser *ad-hoc*, esto es, para un problema concreto. Debido a esto es muy probable que tengamos que repetir gran parte del esfuerzo proyecto a proyecto.

Por este motivo, trabajaremos en un entorno especialmente diseñado para experimentación, en el que gran parte del trabajo ya está realizado y los datos se encuentran procesados en un formato legible y cómodo. Dichos datos nos servirán de *benchmark* para evaluar desde nuestras técnicas de preprocesamiento hasta los modelos que construyamos. Esto permitirá solucionar el problema desde un punto de vista análitico y dejar la integración tecnológica para más adelante.

Concretamente, haremos uso de uno de los formatos más sencillos y expresivos de datos estructurados que existen: `csv`. Un fichero `csv` no es más que un fichero de texto plano donde los datos se almacenan a modo de matriz, en la que cada fila empieza en una nueva línea y cada valor está separado por comas. Al ser ficheros de texto plano se pueden editar con cualquier procesador de texto. Por convención, estos ficheros se guardan con extensión `csv` en lugar de `txt`.

De manera adicional a los datos es muy común incluir una primera fila de cabecera indicando el nombre de cada columna.

---

**Palabras clave**:

* Las filas se denominan **instancias**, **casos** o **ejemplos**.
* Las columnas son **variables** de nuestro problema.

---

El conjunto de datos que vamos a emplear es `iris`. Este fue creado en 1936 por el estadístico y biólogo Ronald Fisher. Se trata de un conjunto de datos muy utilizado, ya que presenta una serie de propiedades que lo hacen ideal para introducirse en este campo. Este contiene 150 muestras tomadas de 3 especies de flores:

* `Iris-setosa`
* `Iris-virginica`
* `Iris-versicolor`

Que conforman los valores de la variable a predecir (`Species`). Para cada una de las flores se han realizado una serie de mediciones correspondientes a las variables predictoras del problema:

* `SepalLengthCm`: Longitud del sépalo (en centímetros)
* `SepalWidthCm`: Anchura del sépalo (en centímetros)
* `PetalLengthCm`: Longitud del pétalo (en centímetros)
* `PetalWidthCm`: Anchura del pétalo (en centímetros)

El objetivo sería clasificar una nueva instancia (cuya especie es desconocida) en función de sus propiedades.

---

**Palabras clave:**

* La variable del conjunto de datos a predecir se denomina **variable objetivo** (t.c.c. **variable clase** en problemas de clasificación).

* El conjunto de **características** o **variables predictoras** es el conjunto de datos sin la variable objetivo (únicamente con las variables que se utilizan para predecirla).

---

Vamos a comenzar cargando el conjunto de datos `iris`:

In [6]:
import pandas as pd

In [7]:
data = pd.read_csv("input/iris/Iris.csv", index_col="Id")

In [8]:
target = "Species"

In [9]:
data[target] = data[target].astype("category")

Nótese que se ha especificado cuál es la variable correspondiente al identificador de casos del conjunto de datos (`Id`) y la variable clase (`Species`).

Una vez hemos cargado el conjunto de datos es fundamental comprobar que el proceso ha funcionado sin problemas, y que las variables y los valores están dentro de lo esperado. Para ello, podemos mostrar las primeras instancias del conjunto de datos o escoger una al azar.

Para obtener las `n` primeras instancias del conjunto de datos podemos usar la función `head`:

In [10]:
data.head(5)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


Esta función es muy útil para comprobaciones rápidas, pero no hay que olvidar que las instancias siempre serán las primeras en orden consecutivo. Esto puede ser un problema si queremos hacer alguna comprobación estadística, ya que la muestra estará sesgada.

Para evitar este problema, lo ideal es obtener una muestra aleatoria del conjunto de datos con `sample`:

--------------

**Palabras clave**:

- Una **muestra sesgada** es aquella que no hemos obtenido con la aleatoriedad suficiente como para que sea representativa de todo el conjunto de datos.

---------------

In [11]:
data.sample(5, random_state=random_state)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
106,7.6,3.0,6.6,2.1,Iris-virginica
133,6.4,2.8,5.6,2.2,Iris-virginica
132,7.9,3.8,6.4,2.0,Iris-virginica
52,6.4,3.2,4.5,1.5,Iris-versicolor
48,4.6,3.2,1.4,0.2,Iris-setosa


Es muy útil disponer del conjunto de datos separado dos subconjuntos, uno con las variables predictoras (`X`) y otro con la variable objetivo (`y`). Se puede utilizar el siguiente fragmento de código para dividirlo: 

In [12]:
X = data.drop(target, axis=1)

In [13]:
y = data[target]

De nuevo, comprobamos que se haya separado correctamente. Comenzamos con las variables predictoras:

In [14]:
X.sample(5, random_state=random_state)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Id,,,,
106,7.6,3.0,6.6,2.1
133,6.4,2.8,5.6,2.2
132,7.9,3.8,6.4,2.0
52,6.4,3.2,4.5,1.5
48,4.6,3.2,1.4,0.2


Y continuamos con la variable clase:

In [15]:
y.sample(5, random_state=random_state)

Id
106     Iris-virginica
133     Iris-virginica
132     Iris-virginica
52     Iris-versicolor
48         Iris-setosa
Name: Species, dtype: category
Categories (3, object): ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

Si bien podríamos comenzar con el análisis exploratorio es fundamental separar nuestro conjunto de datos **como mínimo** en dos:

* Una muestra de entrenamiento (típicamente, 70%)
* Una muestra de prueba (típicamente, 30%)

En un proceso conocido como *holdout*. De este modo, podemos dejar el conjunto de prueba a modo de instancias no observadas para asegurarnos que los resultados de validación han sido estimados de manera honesta (y no optimista). De hecho, si utilizamos el mismo conjunto de datos para aprender y validar un modelo, observaremos un resultado inusual y es que, conforme más sobreajustado está el modelo, menor es el error cometido.

Para realizar un *holdout* podemos utilizar el método `train_test_split` de `scikit-learn`:

---

**Palabras clave:**

* Decimos que un modelo está **sobreajustado** cuando sus parámetros se han aprendido de manera que intentan reproducir perfectamente el conjunto de entrenamiento (de los que conocemos su resultado). Este sobre-entrenamiento (***overfitting***) implica que al modelo le costará más discriminar nuevos casos que presenten datos que nunca han sido observados, presentando un mayor nivel de error en su uso en el mundo real. Cuando un clasificador se aprende de manera que se elimina el sobre-entrenamiento se dice que **generaliza**.

---

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=random_state, train_size=0.7)

Por defecto, las instancias del conjunto de datos se aleatorizan antes de realizar el *holdout*. Esto es muy importante para evitar que, en conjunto de datos ordenados por los valores de la variable clase (como ocurre en `iris`) eliminemos, de uno de los conjuntos, todas las instancias de una (o varias) clase (o clases).

Por otro lado, la semilla se establece mediante el parámetro `random_state`. Este es necesario para que en el proceso de *holdout* siempre se realicen las mismas particiones de entrenamiento y prueba, y así podamos garantizar la reproducibilidad de los experimentos.

Por último, comentar que se ha aplicado un *holdout* estratificado (`stratify=y`), esto es, se ha preservado la proporción de ejemplos de cada clase durante la división. Esto es importante en casos complejos (p.e., problemas desbalanceados) para evitar eliminar gran cantidad de información (aleatorizar el conjunto de datos puede no ser suficiente).

De nuevo, vamos a asegurarnos de que el conjunto de datos se ha dividido correctamente en entrenamiento y prueba. Comenzamos con las variables predictoras del conjunto de datos de entrenamiento:

In [18]:
X_train.sample(5, random_state=random_state)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Id,,,,
49,5.3,3.7,1.5,0.2
33,5.2,4.1,1.5,0.1
66,6.7,3.1,4.4,1.4
63,6.0,2.2,4.0,1.0
111,6.5,3.2,5.1,2.0


Y prueba:

In [19]:
X_test.sample(5, random_state=random_state)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Id,,,,
88,6.3,2.3,4.4,1.3
6,5.4,3.9,1.7,0.4
109,6.7,2.5,5.8,1.8
91,5.5,2.6,4.4,1.2
28,5.2,3.5,1.5,0.2


Por último, finalizamos con la variable objetivo del conjunto de datos de entrenamiento:

In [20]:
y_train.sample(5, random_state=random_state)

Id
49         Iris-setosa
33         Iris-setosa
66     Iris-versicolor
63     Iris-versicolor
111     Iris-virginica
Name: Species, dtype: category
Categories (3, object): ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

Y prueba:

In [21]:
y_test.sample(5, random_state=random_state)

Id
88     Iris-versicolor
6          Iris-setosa
109     Iris-virginica
91     Iris-versicolor
28         Iris-setosa
Name: Species, dtype: category
Categories (3, object): ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

Para facilitar el análisis exploratorio de datos, volvemos a juntar las variables predictoras con la variable clase. Comenzamos con el conjunto de datos de entrenamiento:

In [22]:
data_train = X_train.join(y_train)

Y continuamos con el conjunto de datos de prueba:

In [23]:
data_test = X_test.join(y_test)

Para asegurarnos de que se han juntado correctamente, obtenemos una muestra aleatoria del conjunto de datos de entrenamiento:

In [24]:
data_train.sample(5, random_state=random_state)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
49,5.3,3.7,1.5,0.2,Iris-setosa
33,5.2,4.1,1.5,0.1,Iris-setosa
66,6.7,3.1,4.4,1.4,Iris-versicolor
63,6.0,2.2,4.0,1.0,Iris-versicolor
111,6.5,3.2,5.1,2.0,Iris-virginica


Y del conjunto de datos de prueba:

In [25]:
data_test.sample(5, random_state=random_state)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
88,6.3,2.3,4.4,1.3,Iris-versicolor
6,5.4,3.9,1.7,0.4,Iris-setosa
109,6.7,2.5,5.8,1.8,Iris-virginica
91,5.5,2.6,4.4,1.2,Iris-versicolor
28,5.2,3.5,1.5,0.2,Iris-setosa


# 3. Análisis exploratorio de datos

Antes de comenzar el preprocesamiento es interesante observar las propiedades del conjunto de datos, analizando sus variables y la interacción entre estas. No obstante, no podemos usar el formato tabular directamente puesto que para un humano es casi imposible extraer conclusiones a partir del análisis de valores numéricos. Por ello, nos apoyaremos en gráficos y estadísticos.

### Descripción del conjunto de datos

Antes de realizar cualquier operación es fundamental conocer nuestro problema. Hay dos dimensiones básicas que deben ser exploradas:

* Número de casos
* Número de variables
    * Tipo de las variables: Continuas (t.c.c. numéricas) o discretas (t.c.c. categóricas)
    
Para ello, vamos a recurrir a la clase `ProfileReport` de `pandas_profiling`:

---

**Palabras clave**:

- Una **variable continua** es aquella que se representa mediante valores numéricos y que se distribuye de acuerdo con una función (p.e., distribución normal).

- Una **variable discreta** es aquella compuesta por un conjunto finito de valores (denominados **estados**) a los que se les puede asignar un significado, distribuyéndose de acuerdo a una función probabilística que determina su **frecuencia**.

---

In [26]:
from pandas_profiling import ProfileReport

In [27]:
profile_report = ProfileReport(data_train)

In [28]:
profile_report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Tal y como se puede observar, el conjunto de datos de entrenamiento está formado por 105 casos y 6 variables (1 variable identificadora, 4 variables predictoras y 1 variable clase). Además, el conjunto de datos de entrenamiento no tiene valores perdidos.

Por otro lado, las variables predictoras del conjunto de datos de entrenamiento (`SepalLengthCm`, `SepalWidthCm`, `PetalLengthCm` y `PetalWidthCm`) son numéricas. Por contra, la variable clase (`Species`) es categórica y contiene los siguientes estados:

* `Iris-setosa`
* `Iris-versicolor`
* `Iris-virginica`

Esto es, nuestra variable clase es multivariada con tres estados.

### Visualización de las variables

Una vez conocemos con más detalle el conjunto de datos de entrenamiento, lo que debemos hacer es representar y analizar las distribuciones de las variables. Para ello, utilizaremos métodos univariados, esto es, histogramas para las variables numéricas y diagramas de barras para las variables categóricas. En particular:

* Un histograma muestra la densidad de ejemplos para los distintos valores de una variable numérica.
* Un diagrama de barras representa la frecuencia de cada estado de una variable categórica.

---

**Palabras clave**:

* Un método de análisis **univariado** es aquel que involucra una única variable.

---

Vamos a comenzar visualizando las variables numéricas del conjunto de datos de entrenamiento:

---

**Notas**:

* En el siguiente [enlace](https://plotly.com/python) se proporciona un tutorial de la librería `plotly` con la que ampliar y mejorar los trabajos.

---

In [29]:
import plotly.express as px

In [30]:
px.histogram(X_train)

Tal y como se puede observar, los atributos `SepalLengthCm` y `SepalWidthCm` muestran distribuciones con tendencia central en forma de campana (i.e.,  distribución normal). Por contra, `PetalLengthCm` y `PetalWidthCm` son algo más complejas (i.e., mixtura de distribuciones normales). Lo que también podemos observar es que todos los valores se comportan de manera parecida, sin que podamos observar valores anómalos o ruidosos.

---

**Palabras clave**:

* Un dato **anómalo** (t.c.c. ***outlier***) es aquel que se aleja de la distribución de la variable.
* Un dato **ruidoso** es aquel cuyo valor es erróneo comparado con respecto al resto de valores de la variable.

---

Continuamos visualizando las variables categóricas del problema:

In [31]:
px.histogram(y_train, histnorm="probability")

Lo que podemos observar es que las tres clases de la variable objetivo del problema tienen la misma proporción de casos, esto es, el problema está balanceado.

---

**Palabras clave**:

* Una muestra está **balanceada** con respecto a un conjunto de variables cuando las frecuencias de las distintas combinaciones de estados aparecen en la misma proporción.

---

El análisis univariado nos permite identificar problemas en las variables tales como ruido y *outliers*. También permite detectar distribuciones carentes de información, como pueden ser distribuciones uniformes en las que cada valor es prácticamente único.

No obstante, la mayoría de la información relevante que podemos extraer de un conjunto de datos proviene de estadísticos que se obtienen al contrastar conjunto de variables (análisis multivariado). En problemas de clasificación lo que se busca es determinar la potencia discriminativa de los atributos, para lo cuál se condiciona las relaciones entre estos con respecto a la información que aportan sobre la variable clase.

---

**Palabras clave**:

* Un análisis **multivariado** es aquel que involucra a varias variables del problema.

---

Como caso particular, podemos estudiar relaciones entre pares de variables. Para ello, creamos una matriz de gráficos del tipo nube puntos, en el que cada punto se corresponde con un caso del conjunto de datos y en cada eje se representa un atributo. La información de la variable clase se representa coloreando cada instancia de acuerdo con la clase a la que pertenece:

In [32]:
px.scatter_matrix(data_train, dimensions=X_train, color=target)

Este gráfico muestra una gran cantidad de información relevante para nuestra tarea de clasificación, mostrando el poder discriminativo de cada atributo (diagonal principal de la matriz de gráficos), y para cada combinación de ellos. Tal y como se puede observar, una discretización en 3 intervalos de igual anchura puede ayudar a los algoritmos de aprendizaje a discriminar mejor cada una de las clases.

Por suerte, el conjunto de datos `iris` fue creado con una finalidad didáctica y por ello la tarea de clasificación es muy sencilla y visual. No obstante, no tendremos la misma suerte con otros conjuntos de datos no artificiales, ya que en el mundo real los resultados serán mucho más ruidosos y aparentemente aleatorios.

El análisis visual se realiza en momentos puntuales y para variables concretas una vez que hemos limpiado y analizado nuestros datos. Para empezar es normal utilizar técnicas estadísticas que nos permitan analizar las variables, para lo cual podemos utilizar la librería `pandas_profiling`.

# 4. Preprocesamiento de datos

El preprocesamiento de datos es una de la tareas más importantes del proceso *KDD*, y se estima que debe ocupar, al menos, un 80% del esfuerzo dedicado a un proyecto. Durante esta tarea se transforman los "**datos crudos**" ("***raw data***") en información mucho más accessible por los algoritmos aprendizaje. De hecho, si este contiene demasiados datos redundantes, ruidosos o irrelevantes, el modelo aprendido se va a ajustar a este tipo de datos en lugar de a la información relevante, obteniendo resultados inesperados.

Dentro del preprocesamiento de datos podemos destacar las siguientes tareas:

* Limpieza de datos (imputación de valores perdidos, suavizado del ruido, etc.)
* Integración de datos (a partir de múltiples fuentes)
* Transformación de datos (normalización, construcción, etc.)
* Reducción de datos (discretización de variables numéricas, selección de variables, selección de instancias, etc.)

En el enunciado de esta práctica solo utilizaremos discretización de variables numéricas. No obstante, en el siguiente [enlace](https://scikit-learn.org/stable/modules/preprocessing.html) se proporciona una lista completa de los transformadores que se pueden utilizar para realizar el preprocesamiento.

Para llevar a cabo un buen preprocesamiento es importante conocer el concepto de ***pipeline***. Un *pipeline* no es más que un meta-estimador encargado de aplicar una serie de transformaciones sucesivas a un conjunto de datos antes de aprender y utilizar un modelo.

**¿Porqué usar un *pipeline* en lugar de aplicar las transformaciones deseadas al conjunto de datos?**

Cuando lleguemos a la fase de modelado, no podemos partir de un conjunto de datos ya preprocesado. Esto se debe a que, como típicamente se realiza un particionamiento de este (*holdout*, validación cruzada, etc.) para validar un modelo, estaríamos cometiendo un **fuga de datos** (***data leak***). Esto provoca que las métricas de rendimiento que obtengamos sean demasiado optimistas, pensando que el modelo funciona adecuadamente y bajo los requisitos establecidos. Sin embargo, cuando llegue un conjunto de prueba no visualizado por el algoritmo de aprendizaje, este no va a mostrar el rendimiento deseado dado que no ha sido validado honestamente. Para ello, lo que se debe hacer es utilizar la información del conjunto de entrenamiento para realizar el preprocesamiento tanto en este como en el conjunto de prueba.

**¿Porqué no se aplican las transformaciones al conjunto de entrenamiento y se guardan los valores para aplicarlos al conjunto de prueba?**

El problema es que tendríamos que aplicar todas estas transformaciones manualmente a todos los posibles conjuntos de prueba. Además, este proceso se complica si se utiliza una validación cruzada para obtener las métricas de rendimiento.

**¿Cómo soluciona este problema el *pipeline*?**

Durante el proceso de aprendizaje, al mismo tiempo que el *pipeline* transforma el conjunto de datos de entrenamiento, aprende los parámetros necesarios para realizar posteriores transformaciones utilizando esta misma información. Al final del *pipeline*, nuestro algoritmo de aprendizaje recibe los datos transformados usando únicamente información del conjunto de entrenamiento, y aprende el modelo sobre estos. Por otro lado, al realizar el proceso de inferencia se aplican las mismas transformaciones indicadas en el *pipeline* al conjunto de prueba, pero utilizando los parámetros obtenidos con la información del conjunto de entrenamiento.

---

**Palabras clave**:

* Un **estimador** es un modelo que se utiliza para estimar valores en datos no observados.
* La **imputación de valores perdidos** consiste en rellenar este tipo de datos para que tomen un valor conocido.
* El **modelado** es el proceso mediante el cuál se aprende un modelo (ajuste de sus parámetros) a partir de un conjunto de datos de entrenamiento.
* Se conoce como **datos crudos** a los datos a los que no se les ha realizado ningún tipo de procesamiento.
* Un **fuga de datos** se produce cuando se introduce conocimiento del conjunto de prueba en el conjunto de entrenamiento (p.e., si hemos imputado la media de todo el conjunto de datos en los valores perdidos, y después lo particionamos en entrenamiento y prueba).
* **Validar un modelo** consiste en obtener las predicciones de un conjunto de prueba y obtener métricas de rendimiento conforme a los resultados.

---

### Discretización

Tal y como se ha visto, la discretización permite transformar variables numéricas en categóricas, siendo este paso beneficioso para algunos algoritmos de aprendizaje dado que permite que modelos lineales resuelvan problemas no lineales.

`scikit-learn` permite realizar tres tipos de discretización (`strategy`) mediante el transformador `KBinsDiscretizer`:

* `uniform`: Igual anchura.
* `quantile`: Igual frecuencia.
* `kmeans`: Discretización basada en k-medias.

Tras el análisis exploratorio de datos realizado previamente, parece lógico realizar una discretización en 3 intervalos de igual anchura:

---

**Notas**:

* Obsérvese que solo inicializamos el transformador, sin realizar ningún tipo de transformación sobre los datos.

---

In [33]:
from sklearn.preprocessing import KBinsDiscretizer

In [34]:
k_bins_discretizer = KBinsDiscretizer(n_bins=3, strategy="uniform")

# 5. Algoritmos de clasificación

Antes de empezar a trabajar con algoritmos más potentes de la librería `scikit-learn`, vamos a ver una serie de algoritmos básicos.

### Algoritmo *Zero-R*

El algoritmo *Zero-R* es el más trivial de todos. Básicamente, aprende un clasificador que asigna, a los nuevos casos, la clase predominante en el conjunto de entrenamiento. Este clasificador es inútil en la mayoría de los casos, pero nos servirá como *baseline* a la hora de evaluar la dificultad de un conjunto de datos concreto o la efectividad de un clasificador.

---

**Palabras clave**:

* Un **clasificador** no es más que la instancia específica de un modelo, esto es, un modelo una vez que se ha aprendido a partir del conjunto de datos de entrenamiento, y que puede ser utilizado para obtener predicciones para futuras instancias de entrada.

* Nos referimos como ***baseline*** a un valor trivial en el resultado de clasificación, un punto de partida que puede alcanzarse mediante la observación directa de los datos.

---

Para usar el algoritmo *Zero-R*, recurrimos al estimador `DummyClassifier` de `scikit-learn`:

In [35]:
from sklearn.dummy import DummyClassifier

In [36]:
dummy_model = DummyClassifier(strategy="most_frequent")

Nótese que, para predecir siempre la clase más frecuente en el conjunto de datos de entrenamiento, se debe fijar el hiperparámetro `strategy` a `most_frequent`.

---

**Palabras clave:**

* Un **hiperparámetro** es un parámetro del algoritmo de aprendizaje, que condiciona, de algún modo, cómo se inducirán los parámetros del modelo respecto a los datos. Cuantos más hiperparámetros tenga un algoritmo, mayor complejidad tendrá, pero también será más difícil de configurar el entrenamiento.

---

### Algoritmo *CART* (*Classification and Regression Trees*): Inducción de árboles de decisión

Una vez conocemos el algoritmo *Zero-R* es hora de probar un método más competitivo de la librería `scikit-learn`. Concretamente, utilizaremos un árbol de decisión (aunque solo arañaremos el potencial de la librería en esta práctica).

Para lanzar el algoritmo, usaremos el estimador `DecisionTreeClassifier` de `scikit-learn`:

---

**Notas**:

* Es importante fijar el hiperparámetro `random_state` a una semilla para garantizar la reproducibilidad de los experimentos.

---

In [37]:
from sklearn.tree import DecisionTreeClassifier

In [38]:
decision_tree_model = DecisionTreeClassifier(random_state=random_state)

### *Pipeline*

Para crear un *pipeline*, vamos a usar la función `make_pipeline` de `scikit-learn`. Esta toma como parámetros la lista de transformadores a aplicar al conjunto de datos y, al final de este, el estimador a utilizar.

En particular, vamos a crear un *pipeline* formado por `KBinsDiscretizer` + `DecisionTreeClassifier`, para comparar el rendimiento con respecto al árbol de decisión sin el conjunto de datos discretizado:

In [39]:
from sklearn.pipeline import make_pipeline

In [40]:
pipeline_model = make_pipeline(k_bins_discretizer, decision_tree_model)

# 5. Evaluación de modelos

Ahora es el momento de entrenar y validar nuestros clasificadores. Para ello, vamos a usar una matriz de confusión y un informe de clasificación.

Empezamos con el algoritmo *Zero-R*:

In [41]:
dummy_classifier, dummy_predictions, dummy_report, dummy_matrix = fit_evaluate(dummy_model, X_train, X_test, y_train, y_test)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [42]:
dummy_report

,precision,recall,f1-score,support
Iris-setosa,0.333333,1.000000,0.500000,15.000000
Iris-versicolor,0.000000,0.000000,0.000000,15.000000
Iris-virginica,0.000000,0.000000,0.000000,15.000000
accuracy,0.333333,0.333333,0.333333,0.333333
macro avg,0.111111,0.333333,0.166667,45.000000
weighted avg,0.111111,0.333333,0.166667,45.000000


In [43]:
dummy_matrix

Como era de esperar, el modelo *Zero-R* obtiene malos resultados, pues solo predice la clase mayoritaria en el conjunto de entrenamiento (`Iris-setosa`).

Vamos a ver los resultados del árbol de decisión sin el conjunto de datos discretizado:

In [44]:
decision_tree_classifier, decision_tree_predictions, decision_tree_report, decision_tree_matrix = fit_evaluate(decision_tree_model, X_train, X_test, y_train, y_test)

In [45]:
decision_tree_report

,precision,recall,f1-score,support
Iris-setosa,1.000000,1.000000,1.000000,15.000000
Iris-versicolor,1.000000,0.866667,0.928571,15.000000
Iris-virginica,0.882353,1.000000,0.937500,15.000000
accuracy,0.955556,0.955556,0.955556,0.955556
macro avg,0.960784,0.955556,0.955357,45.000000
weighted avg,0.960784,0.955556,0.955357,45.000000


In [46]:
decision_tree_matrix

Y con el conjunto de datos discretizado:

In [47]:
pipeline_classifier, pipeline_predictions, pipeline_report, pipeline_matrix = fit_evaluate(pipeline_model, X_train, X_test, y_train, y_test)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/base.py:442: UserWarning:

X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names



In [48]:
pipeline_report

,precision,recall,f1-score,support
Iris-setosa,1.000000,1.000000,1.000000,15.000000
Iris-versicolor,1.000000,0.933333,0.965517,15.000000
Iris-virginica,0.937500,1.000000,0.967742,15.000000
accuracy,0.977778,0.977778,0.977778,0.977778
macro avg,0.979167,0.977778,0.977753,45.000000
weighted avg,0.979167,0.977778,0.977753,45.000000


In [49]:
pipeline_matrix

Es evidente que los árboles de decisión obtienen mejores resultados que el algoritmo *Zero-R*. De hecho es importante comentar que el árbol de decisión entrenado con el conjunto de datos discretizado obtiene una mayor tasa de acierto que su homólogo no discretizado.

**¿Cómo ha distinguido el algoritmo de inducción de árboles de decisión las variables categóricas?**

Este algoritmo es capaz de reconocer las variables categóricas gracias al uso de la codificación *one-hot* durante la discretización. Esta codificación  transforma cada atributo categórico con $ n $ estados en $ n $ atributos binarios (uno de ellos fijado a 1 y el resto a 0) indicando a qué estado pertenece la instancia correspondiente.

**¿Por qué se utiliza esta codificación?**

La codificación *one-hot* viene motivada por los algoritmos de aprendizaje que no manejan atributos categóricos, pues permite solucionar elegantemente este problema. De hecho permite obtener modelos más expresivos manteniendo la interpretabilidad.

Por otro lado, la mayor parte de los algoritmos de aprendizaje de la librería `scikit-learn` solo trabajan con variables predictoras numéricas, por lo que aplicar esta transformación es un requisito necesario antes de entrenar los correspondientes modelos. Este es el caso de los árboles de decisión, pues sin necesidad de modificar el algoritmo y usando el método estándar de considerar todos los valores (únicos) como posibles umbrales es capaz de manejar los atributos categóricos adecuadamente.

No obstante, esta codificación no es única sino que `scikit-learn` implementa una gran variedad. La lista completa se encuentra en el siguiente [enlace](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features).

# Trabajo a entregar

Un informe reproduciendo el estudio que se ha hecho con `iris` para `titanic`. En el enlace al conjunto de datos se puede encontrar información sobre el significado semántico del problema. Debéis explorar el conjunto de datos, preprocesarlo y aprender los modelos vistos para después realizar un proceso de clasificación que tendréis que interpretar. El estudio del conjunto de datos se deberá incluir en una única libreta.

Los siguientes elementos deben estar presentes en la libreta:

* Análisis exploratorio de datos mediante gráficas y estadísticos.
* Preprocesamiento de datos.
* Aprendizaje y evaluación de un clasificador *Zero-R*.
* Aprendizaje y evaluación de árboles de clasificación (sin y con preprocesamiento).

Todos los apartados deberán realizarse mediante proceso *holdout* estratificado, discutiendo los resultados y comparando los tres modelos. Además, se deberán analizar las matrices de confusión correspondientes.

Es muy importante tener en cuenta los siguientes criterios de evaluación:

* Deben realizarse todos los puntos anteriores.
* Deben explicarse coherentemente los resultados, utilizando siempre que se pueda un vocabulario técnico (palabras clave).
* El código debe ser claro y estar bien documentado.
* Las explicaciones deben ser claras y el documento estar bien estructurado.
* El código debe ser reproducible.

Como se puede ver, hay 5 puntos de evaluación que supondrán la valoración media de la práctica. Ya sabéis que la media es un estadístico que se ve afectado enormemente por los valores extremos, así que, aunque el contenido sea correcto, si no se explica bien o no está presentable, la nota se verá notablemente afectada.

**Recordar que asociada a esta práctica habrá una entrevista para completar la evaluación de la práctica.**

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9fb9b2ea-858b-4a76-ab52-9a30e37bb453' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>